In [1]:
import subprocess

partitioning = ("small_balanced", "small_random", "mid_balanced", "mid_random", "large_balanced", "large_random")
algorithms = ("fedavg", "fedprox")
communication_rounds = (20, 30)

outputs = {p: {a: {c: None for c in communication_rounds} for a in algorithms} for p in partitioning}

for partition in partitioning:
    for alg in algorithms:
        for rounds in communication_rounds:
            command = f"python experiments.py --dataset=clinical --model=mlp --device=cpu --comm_round={rounds} --n_parties=10 --partition={partition} --alg={alg}"
            print(f"Executing `{command}`")
            process = subprocess.Popen(command.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            _, output = process.communicate()
            outputs[partition][alg][rounds] = output
            break
        break
    break


Executing `python experiments.py --dataset=clinical --model=mlp --device=cpu --comm_round=20 --n_parties=10 --partition=small_balanced --alg=fedavg`


In [4]:
print(outputs["small_balanced"]["fedavg"][20].decode())

04-10 19:23 INFO     cpu
04-10 19:23 INFO     ####################################################################################################
04-10 19:23 INFO     Partitioning data
04-10 19:23 INFO     Column f01 => replacement = 74.65631621841891 | 15.05 .. 300.06
04-10 19:23 INFO     Column f02 => replacement = 13.766148325358852 | 3.0 .. 15.0
04-10 19:23 INFO     Column f03 => replacement = 150.1475458861351 | 81.0 .. 323.0
04-10 19:23 INFO     Column f04 => replacement = 92.60691637697121 | 0.300000012 .. 180.0
04-10 19:23 INFO     Column f05 => replacement = 37.536022712539456 | 31.6 .. 42.77777778
04-10 19:23 INFO     Column f06 => replacement = 236.63519941025135 | 27.0 .. 28166.66667
04-10 19:23 INFO     Column f07 => replacement = 1.588914748728558 | 0.1 .. 79.0
04-10 19:23 INFO     Column f08 => replacement = 2028.5945181659674 | -1730.0 .. 51520.0
04-10 19:23 INFO     Column f09 => replacement = 13.936415200299512 | 0.1 .. 600.2
04-10 19:23 INFO     Column f10 => replac

In [ ]:
import re


def extract_test_history(outputs_: dict[str, dict[int, dict[str, bytes]]]) -> dict[
    str, dict[int, dict[str, list[float]]]]:
    result = {p: {a: {c: [] for c in communication_rounds} for a in algorithms} for p in partitioning}
    regex = "net \d+ final test acc (\d+[.]\d+)"
    for partition in partitioning:
        for alg in algorithms:
            for rounds in communication_rounds:
                snippet = outputs_[partition][alg][rounds].decode().split("in comm round:")
                for subsnippet in snippet[1:]:
                    result[partition][alg][rounds].append(
                        re.findall(
                            regex,
                            subsnippet
                        )
                    )
    return result

In [ ]:
extract_test_history(outputs)